In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

### One a Day

In [ ]:
# load data from csv
df = pd.read_csv('nsadata/logins.txt', sep='\t', header=None, names=['date', 'time', 'user', 'way'])
df

In [ ]:
# need to find entries that have multiple logins (WAY = "IN") in the same day
# group by date and user, and count the number of logins greater than 1
df_new = df[df['way'] == 'IN'].groupby(['date', 'user']).size().reset_index(name='count')

# count greater than 1 and user stars with 'e.'
df_new[(df_new['count'] > 1) & (df_new['user'].str.startswith('e.'))]['user'].iloc[0]

### shifty

In [ ]:
# People have normal schedules of when they like to show up to work.

# Find an account that starts with j. of someone who sometimes works a different shift than their normal.

# we can do this by grouping by user, getting median time, and then finding the user who has a different time
# than the median
# get second of day
df['time_second'] = pd.to_datetime(df['time']).dt.hour * 3600 + pd.to_datetime(df['time']).dt.minute * 60 + pd.to_datetime(df['time']).dt.second
df_new = df[df['way']=='IN'].groupby('user')['time_second'].apply(lambda x: x.median())#.reset_index(name='median_time')


df_in = df[(df['way']=='IN') & (df['user'].str.startswith('j.'))] 
# add a column for distance from median and then sort and get highest
df_in['time_diff'] = abs(df_in['time_second'] - df_in['time_second'].median())

# calculate standard deviation for every user and then find the user with the highest standard deviation
df_in.groupby('user')['time_second'].std().sort_values(ascending=False).index[0]
# 'j.salano'

In [ ]:
df[(df['user']=='j.salano' )& (df['way']=='IN')]['time_second'].hist(bins=30)

### retirement home

In [ ]:
# In this dataset, some people are hired and others retire. There is no log letting you know that someone has retired. However, retired people should never log in again.

# Can you find the usernames that appear to log in after they've retired? Please input the username that starts with m..


# group by user then compare the difference in day (retrieved from date) between the next row
# then find row where the difference is greater than 100
df_ms = df[df['user'].str.startswith('m.')]
df_ms['date'] = pd.to_datetime(df_ms['date'])
df_ms['date_diff'] = df_ms.groupby('user')['date'].diff().dt.days
df_ms[df_ms['date_diff'] > 20]['user']

### traffic jam

In [ ]:
# In this dataset, traffic is simulated in 5 zones of the city: north, east, south, west, and central. User s.kinkel lives in the west zone.

# **Which day of the year, in YYYY-MM-DD format, did the west zone have the worst traffic? **

# Hint for the traffic jam: There's enough data for you to calculate the average start times of most employees. Then you can start to cluster the employees based on groups that exhibit delays on the same day (since traffic affects all the users in the same zone). From there, you can find the cluster that s.kinkel groups with and find out when that group had the worst traffic.


# Note that s.kinkel may not have worked that day.
# we can groupby date and find the count and then sort by count descending
# first find dates when s.kinkel worked
df_s = df[df['user'] == 's.kinkel']
# then get all
df_west = df[df['user'] != 's.kinkel']
df_west['date'] = pd.to_datetime(df_west['date'])
df_west.groupby('date').size().sort_values(ascending=False)

In [ ]:
# histogram of time of day and log in counts
df_in = df[df['way']=='IN']
df_in['time'] = pd.to_datetime(df_in['time'])
df_in['time_second'] = df_in['time'].dt.hour * 3600 + df_in['time'].dt.minute * 60 + df_in['time'].dt.second

df_in['time_second'].hist(bins=80)

### hire me

In [ ]:
# How many people were hired throughout the year (the entire dataset)?

# count unique users and subtract users whose first 'way' is 'OUT'
unique = df['user'].nunique()

# how many people's first traffic was 'OUT'
# group by user and get first way
df_first = df.groupby('user').first()
logout = df_first[df_first['way'] == 'OUT'].shape[0]
# logout
unique-logout

In [ ]:
df_first[df_first['way'] == 'OUT']

### timecard

In [ ]:
# There's one account that changed behavior in July in a way that is hard to detect. The employee is cheating the timecard system.

# maybe they signed in around the same time with some variance which changed after July

# split data into before and after July
df['date'] = pd.to_datetime(df['date'])
df['time'] = pd.to_datetime(df['time'])
df['seconds'] = df['time'].dt.hour * 3600 + df['time'].dt.minute * 60 + df['time'].dt.second
df_before = df[df['date'] < '2021-07-01']
df_after = df[df['date'] >= '2021-07-01']

# group by user and get std way=IN time
df_before_in = df_before[df_before['way'] == 'IN'].groupby('user')['seconds'].std()
df_after_in = df_after[df_after['way'] == 'IN'].groupby('user')['seconds'].std()

# find user with the highest change in std after July
(df_after_in).sort_values(ascending=True)

In [ ]:
df_after[df_after['user'] == 'd.tye']